In [29]:
%%writefile mapper4.py

import sys
import re

def read_stopwords(file_path):
    return set(word.strip() for word in open(file_path))

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stop_words = read_stopwords('stop_words_en.txt')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.strip()
        if word and len(word) > 1 and word.isalpha():
            if word.lower() in stop_words:
                continue
            wordList = list(word.lower())
            wordset = set(wordList)
            if len(wordset) > 1:
                wordList.sort()
                print ("%s\t%s\t%d" % (''.join(wordList), word.lower(), 1))
            

Overwriting mapper4.py


In [17]:
%%writefile reducer4.py

import sys

current_anagram = None
current_word = None
word_count = 0
a_count = 0


for line in sys.stdin:
    try:
        anagram, word, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    if current_word == word:
        word_count += count
    else:
        if current_anagram:
            if current_word:
                print ("%s\t%s\t%d" % (current_anagram, current_word, word_count))
        word_count = count
        current_anagram = anagram
        current_word = word

if current_anagram and current_word:
    print ("%s\t%s\t%d" % (current_anagram, current_word, word_count))

Overwriting reducer4.py


In [24]:
%%writefile reducer4a.py

import sys

current_anagram = None
current_word = None
total_count = 0
word_count = 0
word_list = []


for line in sys.stdin:
    try:
        anagram, word, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    #print("--- %s\t%s\t%d" % (anagram, word, count))
    #print("*** %s\t%s" % (current_anagram, current_word))
    if current_anagram and current_anagram == anagram and current_word == word:
        total_count += count
        continue
    if current_anagram and current_anagram == anagram:
        word_list.append(word)
        #print(','.join(word_list))w
        total_count += count    
        word_count += 1
        current_word = word
    else:
        if current_anagram:
            if current_word and word_count > 1:  
                word_set = set(word_list) 
                if len(word_set):
                    word_list = list(word_set)
                    word_list.sort()
                    print ("%d\t%d\t%s" % (total_count, len(word_list), ','.join(word_list)))
        word_count = 1
        total_count = count
        word_list = []
        word_list.append(word)
        current_anagram = anagram
        current_word = word

if current_anagram and current_word:
    word_list.sort()
    print ("%d\t%d\t%s" % (total_count, word_count, ','.join(word_list)))

Overwriting reducer4a.py


In [35]:
%%writefile mapper4b.py

import sys
import re

#reload(sys)
#sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        total_count, word_count, str = line.strip().split('\t', 2)
        total_count = int(total_count)
        word_count = int(word_count)
    except ValueError as e:
        continue
    if word_count > 1:
        print ("%d\t%d\t%s" % (total_count, word_count, str))



Overwriting mapper4b.py


In [5]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [30]:
%%bash

OUT_DIR="wc_result_"$(date +"%s%6N")
NUM_REDUCERS=4
OUT_DIR1="anagram_int_"$(date +"%s%6N")
OUT_DIR2="anagram_result_"$(date +"%s%6N")

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options=-k1,2 \
    -files mapper4.py,reducer4.py,/datasets/stop_words_en.txt \
    -mapper "python mapper4.py" \
    -reducer "python reducer4.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 
    

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Anagrams" \
    -D mapreduce.job.reduces=${NUM_REDICERS} \
    -files reducer4a.py \
    -mapper "cat" \
    -reducer "python reducer4a.py" \
    -input ${OUT_DIR}/part* \
    -output ${OUT_DIR1} > /dev/null 


yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Anagrams final" \
    -D mapreduce.job.reduces=1 \
    -files mapper4b.py \
    -mapper "python mapper4b.py" \
    -input ${OUT_DIR1}/part* \
    -reducer "cat" \
    -output ${OUT_DIR2} > /dev/null 

hdfs dfs -cat ${OUT_DIR2}/part-00000 | grep "english,"